In [1]:
# Cell 1: Import libraries
import sqlite3
import pandas as pd
import json

# Cell 2: Connect to database
DB_PATH = "news_brief_01.db"
conn = sqlite3.connect(DB_PATH)

# Cell 3: View all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table'"
tables_df = pd.read_sql_query(tables_query, conn)
print("📊 Available Tables:")
display(tables_df)

# Cell 4: View cached_news table summary
summary_query = """
    SELECT 
        category,
        date,
        json_array_length(articles) as article_count,
        created_at
    FROM cached_news
    ORDER BY date DESC, category
"""
summary_df = pd.read_sql_query(summary_query, conn)
print(f"\n📰 Cached News Summary ({len(summary_df)} records):")
display(summary_df)

# Cell 5: View specific category data
tech_news = pd.read_sql_query("""
    SELECT category, date, articles 
    FROM cached_news 
    WHERE category = 'Technology' 
    ORDER BY date DESC
""", conn)
print("\n💻 Technology News:")
display(tech_news)

# Cell 6: Extract and view actual articles from JSON
def extract_articles(row):
    """Extract articles from JSON column"""
    articles = json.loads(row['articles'])
    return pd.DataFrame(articles)

# Get latest tech news articles
if len(tech_news) > 0:
    latest_tech = tech_news.iloc[0]
    articles_df = extract_articles(latest_tech)
    print(f"\n📄 Articles for {latest_tech['category']} on {latest_tech['date']}:")
    display(articles_df[['title', 'source', 'url']])

# Cell 7: Statistics and visualizations
stats_query = """
    SELECT 
        date,
        SUM(json_array_length(articles)) as total_articles,
        COUNT(DISTINCT category) as categories_count
    FROM cached_news
    GROUP BY date
    ORDER BY date DESC
"""
stats_df = pd.read_sql_query(stats_query, conn)
print("\n📈 Daily Statistics:")
display(stats_df)

# Cell 8: Category-wise breakdown
category_query = """
    SELECT 
        category,
        COUNT(*) as days_with_data,
        SUM(json_array_length(articles)) as total_articles,
        AVG(json_array_length(articles)) as avg_articles_per_day
    FROM cached_news
    GROUP BY category
    ORDER BY total_articles DESC
"""
category_df = pd.read_sql_query(category_query, conn)
print("\n📂 Category Breakdown:")
display(category_df)

# Cell 9: Search for specific keywords
def search_news(keyword, conn):
    """Search for news articles containing keyword"""
    query = """
        SELECT category, date, articles 
        FROM cached_news 
        WHERE json_extract(articles, '$') LIKE ?
    """
    results = pd.read_sql_query(query, conn, params=[f'%{keyword}%'])
    
    all_matching = []
    for _, row in results.iterrows():
        articles = json.loads(row['articles'])
        for article in articles:
            if keyword.lower() in article['title'].lower():
                all_matching.append({
                    'category': row['category'],
                    'date': row['date'],
                    'title': article['title'],
                    'source': article['source'],
                    'url': article['url']
                })
    
    return pd.DataFrame(all_matching)

# Example search
search_results = search_news("AI", conn)
print(f"\n🔍 Search results for 'AI' ({len(search_results)} articles):")
display(search_results.head(10))

# Cell 10: Close connection
conn.close()
print("\n✅ Database connection closed")


📊 Available Tables:


,name
0,cached_news
1,sqlite_sequence



📰 Cached News Summary (84 records):


,category,date,article_count,created_at
0,Business,2026-02-05,15,2026-02-05 14:30:37
1,Entertainment,2026-02-05,14,2026-02-05 14:30:57
2,Health,2026-02-05,15,2026-02-05 14:30:50
3,Politics,2026-02-05,15,2026-02-05 14:31:04
4,Sports,2026-02-05,15,2026-02-05 14:30:43
...,...,...,...,...
79,Entertainment,2026-01-23,0,2026-02-05 14:38:52
80,Health,2026-01-23,0,2026-02-05 14:38:46
81,Politics,2026-01-23,0,2026-02-05 14:38:58
82,Sports,2026-01-23,0,2026-02-05 14:38:40



💻 Technology News:


...


📂 Category Breakdown:


,category,days_with_data,total_articles,avg_articles_per_day
0,Business,14,195,13.928571
1,Sports,14,195,13.928571
2,Technology,14,195,13.928571
3,Health,14,180,12.857143
4,Politics,14,180,12.857143
5,Entertainment,14,179,12.785714



🔍 Search results for 'AI' (289 articles):


,category,date,title,source,url
0,Technology,2026-02-05,"Explained - Why Nvidia, Broadcom shares rose a...",CNBC TV18,https://www.cnbctv18.com/market/nvidia-broadco...
1,Technology,2026-02-05,TSMC plans major upgrade of Japan chip plant i...,The Japan Times,https://www.japantimes.co.jp/business/2026/02/...
2,Technology,2026-02-05,Energy and critical minerals in focus as Jaish...,The Indian Express,https://indianexpress.com/shorts/india/energy-...
3,Technology,2026-02-05,X offices raided in France as UK opens fresh i...,BBC News,https://www.bbc.com/news/articles/ce3ex92557jo...
4,Business,2026-02-05,US households become increasingly strained in ...,Times of India,https://timesofindia.indiatimes.com/business/i...
5,Business,2026-02-05,OpenAI is hiring hundreds of 'forward deployed...,MarketScreener,https://www.marketscreener.com/news/openai-is-...
6,Business,2026-02-05,"Numerology Number 9 Prediction Today, February...",India Today,https://www.indiatoday.in/horoscopes/numerolog...
7,Business,2026-02-05,The 'striking silence' that convinced police ...,BBC News,https://www.bbc.com/news/articles/c5yv1d7g2ldo...
8,Business,2026-02-05,Warning of long airport queues under new EU bo...,BBC News,https://www.bbc.com/news/articles/cn0k699pxwzo...
9,Business,2026-02-05,Why Target is under fire over Minnesota ICE raids,BBC News,https://www.bbc.com/news/articles/c4g4y4gwjpeo...



✅ Database connection closed


In [2]:
# Cell 1: Import libraries
import sqlite3
import pandas as pd
import json

# Cell 2: Connect to database
DB_PATH = "news_brief.db"
conn = sqlite3.connect(DB_PATH)

# Cell 3: View all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table'"
tables_df = pd.read_sql_query(tables_query, conn)
print("📊 Available Tables:")
display(tables_df)

# Cell 4: View cached_news table summary
summary_query = """
    SELECT 
        category,
        date,
        json_array_length(articles) as article_count,
        created_at
    FROM cached_news
    ORDER BY date DESC, category
"""
summary_df = pd.read_sql_query(summary_query, conn)
print(f"\n📰 Cached News Summary ({len(summary_df)} records):")
display(summary_df)

# Cell 5: View specific category data
tech_news = pd.read_sql_query("""
    SELECT category, date, articles 
    FROM cached_news 
    WHERE category = 'Technology' 
    ORDER BY date DESC
""", conn)
print("\n💻 Technology News:")
display(tech_news)

# Cell 6: Extract and view actual articles from JSON
def extract_articles(row):
    """Extract articles from JSON column"""
    articles = json.loads(row['articles'])
    return pd.DataFrame(articles)

# Get latest tech news articles
if len(tech_news) > 0:
    latest_tech = tech_news.iloc[0]
    articles_df = extract_articles(latest_tech)
    print(f"\n📄 Articles for {latest_tech['category']} on {latest_tech['date']}:")
    display(articles_df[['title', 'source', 'url']])

# Cell 7: Statistics and visualizations
stats_query = """
    SELECT 
        date,
        SUM(json_array_length(articles)) as total_articles,
        COUNT(DISTINCT category) as categories_count
    FROM cached_news
    GROUP BY date
    ORDER BY date DESC
"""
stats_df = pd.read_sql_query(stats_query, conn)
print("\n📈 Daily Statistics:")
display(stats_df)

# Cell 8: Category-wise breakdown
category_query = """
    SELECT 
        category,
        COUNT(*) as days_with_data,
        SUM(json_array_length(articles)) as total_articles,
        AVG(json_array_length(articles)) as avg_articles_per_day
    FROM cached_news
    GROUP BY category
    ORDER BY total_articles DESC
"""
category_df = pd.read_sql_query(category_query, conn)
print("\n📂 Category Breakdown:")
display(category_df)

# Cell 9: Search for specific keywords
def search_news(keyword, conn):
    """Search for news articles containing keyword"""
    query = """
        SELECT category, date, articles 
        FROM cached_news 
        WHERE json_extract(articles, '$') LIKE ?
    """
    results = pd.read_sql_query(query, conn, params=[f'%{keyword}%'])
    
    all_matching = []
    for _, row in results.iterrows():
        articles = json.loads(row['articles'])
        for article in articles:
            if keyword.lower() in article['title'].lower():
                all_matching.append({
                    'category': row['category'],
                    'date': row['date'],
                    'title': article['title'],
                    'source': article['source'],
                    'url': article['url']
                })
    
    return pd.DataFrame(all_matching)

# Example search
search_results = search_news("AI", conn)
print(f"\n🔍 Search results for 'AI' ({len(search_results)} articles):")
display(search_results.head(10))

# Cell 10: Close connection
conn.close()
print("\n✅ Database connection closed")


📊 Available Tables:


,name


DatabaseError: Execution failed on sql '
    SELECT 
        category,
        date,
        json_array_length(articles) as article_count,
        created_at
    FROM cached_news
    ORDER BY date DESC, category
': no such table: cached_news